**Linn Cao Nguyen Phuong**

Spring 2023

2/21/2023

CS 421

Project 2: Python2Vec

In [2]:
import numpy as np
import pandas as pd

In [4]:
pip install gensim

  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached Cython-0.29.32-py2.py3-none-any.whl (986 kB)
  Using cached FuzzyTM-2.0.5-py3-none-any.whl (29 kB)
  Using cached pyFUME-0.2.25-py3-none-any.whl (67 kB)
  Using cached fst-pso-1.8.1.tar.gz (18 kB)
  Using cached simpful-2.9.0-py3-none-any.whl (30 kB)
  Using cached miniful-0.0.6.tar.gz (2.8 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
Using legacy 'setup.py install' for fst-pso, since package 'wheel' is not installed.
Using legacy 'setup.py install' for miniful, since package 'wheel' is not installed.
    Running setup.py install for miniful: started
    Running setup.py install for miniful: finished with status 'done'
    Running setup.py install for fst-pso: started
    Running setup.py install for fst-pso: finished with status 'don

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\linnxinh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [27]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\linnxinh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import gensim
import os

In [23]:
py_files = []
for dirpath, dirnames, filenames in os.walk('libraries'):
    for filename in filenames:
        if filename.endswith('.py'):
            py_files.append(os.path.join(dirpath,filename))

# print(py_files[:10])

In [24]:
with open('py_files.txt', 'w', encoding='utf-8', errors='replace') as f:
    for py_file in py_files:
        try:
            with open(py_file, 'r', encoding='utf-8', errors='replace') as f_in:
                f.write(f_in.read() + '\n')
        except OSError:
            continue

In [33]:
import codecs
import string
from gensim.models import Word2Vec

def tokenize_line(line):
    line = line.lower()  
    word_list = [word.strip(string.punctuation) for word in line.split()]
    word_list = list(filter(None, word_list))
    return word_list

with codecs.open('py_files.txt', 'r', 'utf-8') as f:
    lines_of_code = []
    num_lines = 0
    tokens = 0
    for line in f:
        if line.strip().startswith("#"):
            continue  
        lines_of_code.append(tokenize_line(line))
        num_lines += 1
        tokens += len(tokenize_line(line))

print("Number of lines:", num_lines)
print("Tokens:", tokens)


Number of lines: 4606201
Tokens: 11053140


In [37]:
model = Word2Vec(lines_of_code, window=5, min_count=1)

print("Closest words to \"for\":")
print(model.wv.most_similar("for"))

print("\nClosest words to \"if\":")
print(model.wv.most_similar("if"))

print("\nSimilarity of \"math\" and \"numpy\":")
print(model.wv.similarity("math", "numpy"))

Closest words to "for":
[('in', 0.7395472526550293), ('general', 0.6883535981178284), ('etc', 0.6875367164611816), ('borg', 0.6808226108551025), ('numerical', 0.6750637292861938), ('selection', 0.6706938743591309), ('including', 0.6690239310264587), ('below', 0.6681692004203796), ('within', 0.6670719981193542), ('matching', 0.667030394077301)]

Closest words to "if":
[('elif', 0.7820208072662354), ('not', 0.7155658006668091), ('and', 0.6704928874969482), ('both', 0.6607910990715027), ('force', 0.6586593389511108), ('auto', 0.6574268341064453), ('else', 0.6554098725318909), ('otherwise', 0.6551583409309387), ('enabled', 0.6409156918525696), ('then', 0.6270421743392944)]

Similarity of "math" and "numpy":
0.5131376


In [55]:
import ast
from collections import Counter

with codecs.open('py_files.txt', "r", "utf-8") as f:
    code = f.read()

# get a list of all identifiers in the ast tree
identifiers = []
for line in code.splitlines():
    try:
        ast_tree = ast.parse(line)
        for node in ast.walk(ast_tree):
            if isinstance(node, ast.Name):
                identifiers.append(node.id)
    except SyntaxError:
        continue

In [59]:
# count the frequency
freq = Counter(identifiers)

# find the 50 most common identifiers 
with codecs.open('most_common.txt', 'w', encoding='utf-8', errors='replace') as f:
    for name, count in freq.most_common(50):
        f.write("Name: " + str(name) + ", count: " + str(count) + "\n")

In [64]:
import itertools

common_ids = [name for name, count in freq.most_common(50)]

similarities = {}
for id1, id2 in itertools.combinations(common_ids, 2):
    try:
        similarity = model.wv.similarity(id1, id2)
        similarities[(id1, id2)] = similarity
    except KeyError:
        continue

print('Most similar pairs:')
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
for pair, similarity in sorted_similarities[:10]:
    print(f'{pair[0]} - {pair[1]}: {similarity:.4f}')

print('\nLeast similar pairs:')
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1])
for pair, similarity in sorted_similarities[:10]:
    print(f'{pair[0]} - {pair[1]}: {similarity:.4f}')

Most similar pairs:
ax1 - ax2: 0.9884
axs - ax2: 0.9662
axs - ax1: 0.9617
fig - axs: 0.9491
fig - ax1: 0.9359
fig - ax2: 0.9336
re - sys: 0.9237
np - plt: 0.9222
re - pytest: 0.8869
plt - default_manager: 0.8746

Least similar pairs:
def_gen - data: -0.1764
x - def_gen: -0.1498
y - def_gen: -0.0928
type - ax2: -0.0636
ax2 - s: -0.0496
str - clf: -0.0474
int - clf: -0.0402
type - clf: -0.0382
i8 - time: -0.0371
fig - s: -0.0370
